In [2]:
import pandas as pd
import re, string
import nltk
import torch
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset

# Sadece gerekli olan NLTK kaynaklarını indiriyoruz
#nltk.download('stopwords')
#nltk.download('wordnet')

c:\Users\sagla\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# CSV dosyasını oku (aynı klasörde olduğunu varsayalım)
df = pd.read_csv('IMDB Dataset.csv')
print("Veri kümesi yüklendi.")
print(df.head())

Veri kümesi yüklendi.
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [4]:
import re

def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

# Apply preprocessing
df['cleaned_review'] = df['review'].apply(preprocess_text)

In [33]:
def clean_text_basic(text):
    # Küçük harf yap
    text = text.lower()
    
    # HTML etiketlerini kaldır
    text = re.sub(r'<.*?>', '', text)
    
    # URL'leri kaldır
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # Noktalama işaretlerini kaldır
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Emoji temizleme
    emoji_pattern = re.compile(
        "[" u"\U0001F600-\U0001F64F" u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF" u"\U0001F1E0-\U0001F1FF"
        u"\U00002702-\U000027B0" "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    return text

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# POS bilgisi olmadan varsayılan isim (noun) kabul ederek sade lemmatization
def lemmatize_simple(text):
    words = text.split()  # burada tokenizer yerine split kullanıyoruz
    cleaned = [
        lemmatizer.lemmatize(word) for word in words
        if word.lower() not in stop_words and word.isalpha()
    ]
    return ' '.join(cleaned)

# Full temizlik + lemmatizasyon
def full_clean(text):
    text = clean_text_basic(text)
    text = lemmatize_simple(text)
    return text

# Uygula
df['cleaned_review'] = df['review'].apply(full_clean)

In [ ]:
slang_dict = {"omg": "oh my god", "lol": "laugh out loud", "u": "you", "r": "are", "gr8": "great", "b4": "before", "asap": "as soon as possible", "btw": "", "fyi": "for your information", "idk": "i don't know", "imo": "in my opinion", "jk": "just kidding", "lmao": "laughing my ass off", "lmk": "let me know", "nvm": "nevermind", "np": "no problem", "rofl": "rolling on the floor laughing", "smh": "shaking my head", "tbh": "to be honest", "thx": "thanks", "ty": "thank you", "wth": "what the heck", "wtf": "what the f***", "yolo": "you only live once", "brb": "be right back", "gtg": "got to go", "btw": "by the way", "ttyl": "talk to you later", "ily": "I love you", "g2g": "got to go", "h8": "hate", "jk": "just kidding", "thx": "thanks", "ttyl": "talk to you later", "cya": "see you", "gg": "good game", "afk": "away from keyboard", "ez": "easy", "wb": "welcome back", "idc": "i don't care", "rn": "right now", "lmk": "let me know", "ikr": "i know right", "tmi": "too much information", "smh": "shaking my head", "w/e": "whatever", 
                 "bff": "best friends forever", "tfw": "that feeling when", "np": "no problem", "nvm": "nevermind", "fyi": "for your information", "cuz": "because", "gonna": "going to", "wanna": "want to", "gotta": "got to", "kinda": "kind of", "sorta": "sort of", "dunno": "don't know", "ain't": "is not", "gimme": "give me", "lemme": "let me", "im": "I'm", "hes": "he's", "shes": "she's", "theyre": "they're", "youre": "you're", "havent": "have not", "cant": "cannot",
                  "couldnt": "could not", "didnt": "did not", "doesnt": "does not", "dont": "do not", "isnt": "is not", "mightnt": "might not", "mustnt": "must not", "shouldnt": "should not", "wasnt": "was not", "werent": "were not", "wouldnt": "would not", "tho": "though", "thru": "through", "nite": "night", "neva": "never", "sum": "some", "dat": "that", "dis": "this", "dem": "them", "dey": "they", "em": "them", "whered": "where did", "whod": "who did", "whos": "who's", "yall": "you all", "dam": "damn", "hell": "hell", "pissed": "pissed", "fck": "fuck", "fuk": "fuck", "effing": "effing", "bs": "bullshit", "crap": "crap", "shitty": "shitty", "wtf": "what the fuck", "stfu": "shut the fuck up", "gtfo": "get the fuck out", "irl": "in real life", "roflmao": "rolling on the floor laughing my ass off", "smol": "small", "big oof": "huge mistake", "yeet": "throw something forcefully", "pog": "play of the game", "sus": "suspicious", "cap": "lie", "no cap": "no lie", "bet": "okay", "lit": "amazing", "fr": "for real", "bruh": "bro", "fam": "family",
                    "goat": "greatest of all time", "lowkey": "somewhat", "highkey": "very", "vibe": "mood", "drip": "fashionable", "slay": "do something well", "tea": "gossip", "sksksk": "excited reaction", "tf": "the fuck", "fomo": "fear of missing out", "tldr": "too long didn't read", "hmu": "hit me up", "wyd": "what you doing", "wym": "what you mean", "wdym": "what do you mean", "btw": "by the way", "imo": "in my opinion", "idc": "i don't care", 
                  "idgaf": "i don't give a f***", "frfr": "for real for real", "mf": "motherfucker", "rn": "right now", "lolz": "laughs", "hmu": "hit me up", "hbu": "how about you", "rn": "right now", "lmk": "let me know", "ikr": "i know right", "jk": "just kidding", "fr": "for real", "ffs": "for fuck's sake", "btw": "by the way", "nfs": "not for sale", "dm": "direct message", "af": "as fuck", "idk": "i don't know"}

def correct_slangs(text, slang_dict):
    for slang, replacement in slang_dict.items():
        text = re.sub(rf'\b{slang}\b', replacement, text)
    return text

# Slang düzeltmeyi orijinal review'e uygula (henüz temizlenmemiş)
df['cleaned_review'] = df['cleaned_review'].apply(lambda x: correct_slangs(x.lower(), slang_dict))

In [15]:
df.to_csv("cleaned_IMDB.csv")

In [25]:
import pandas as pd
df=pd.read_csv("cleaned_IMDB.csv")

In [5]:
df = df.dropna(subset=['cleaned_review', 'sentiment'])

In [6]:
import numpy as np
from sklearn.model_selection import train_test_split

# Etiketleri sayıya çevir
df['label'] = df['sentiment'].map({'positive': 1, 'negative': 0})

# Özellik ve etiketleri ayır
X = df['cleaned_review'].values
y = df['label'].values

# %80 train, %20 test
x_train, x_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

# Kontrol
print("Veri şekilleri:")
print(f"Train: {x_train.shape}, Test: {x_test.shape}")

Veri şekilleri:
Train: (40000,), Test: (10000,)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score

# TfidfVectorizer (Binary + max 5000 feature)
vectorizer = TfidfVectorizer(max_features=5000)

x_train_vec = vectorizer.fit_transform(x_train)
x_test_vec = vectorizer.transform(x_test)

# Model eğitme ve değerlendirme fonksiyonu
def train_and_evaluate_model(model, model_name):
    print(f"\n🔹 {model_name}")
    model.fit(x_train_vec, y_train)
    preds = model.predict(x_test_vec)
    # Skorlar
    acc = accuracy_score(y_test, preds)
    f1 = f1_score(y_test, preds)

    print(f"Accuracy: {acc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    

In [15]:
# Logistic Regression
train_and_evaluate_model(LogisticRegression(max_iter=1000), "Logistic Regression")


🔹 Logistic Regression
Accuracy: 0.8932
F1 Score: 0.8953


In [6]:
# KNN
train_and_evaluate_model(KNeighborsClassifier(n_neighbors=3), "K-Nearest Neighbors")


🔹 K-Nearest Neighbors
K-Nearest Neighbors - Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.77      0.65      4961
           1       0.65      0.41      0.50      5039

    accuracy                           0.59     10000
   macro avg       0.60      0.59      0.58     10000
weighted avg       0.61      0.59      0.58     10000

K-Nearest Neighbors - Accuracy on Training Set: 0.5896


In [17]:
# Random Forest
train_and_evaluate_model(RandomForestClassifier(n_estimators=100), "Random Forest")


🔹 Random Forest
Accuracy: 0.8485
F1 Score: 0.8480


In [18]:
# Linear Support Vector Classifier (SVC'den çok daha hızlıdır)
from sklearn.svm import LinearSVC
train_and_evaluate_model(LinearSVC(), "Linear SVC")


🔹 Linear SVC
Accuracy: 0.8868
F1 Score: 0.8888


In [16]:
# Multinomial Naive Bayes
train_and_evaluate_model(MultinomialNB(), "Multinomial Naive Bayes")


🔹 Multinomial Naive Bayes
Accuracy: 0.8508
F1 Score: 0.8516


In [ ]:
# SVM
train_and_evaluate_model(SVC(), "SVM")


🔹 SVM


In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(x_train)
X_train_sequences = tokenizer.texts_to_sequences(x_train)
X_test_sequences = tokenizer.texts_to_sequences(x_test)

# Pad sequences
max_sequence_length = max(len(seq) for seq in X_train_sequences)
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length)


In [9]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.callbacks import EarlyStopping

cnn_model = Sequential()
cnn_model.add(Embedding(input_dim=5000, output_dim=128, input_length=max_sequence_length))
cnn_model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dense(1, activation='sigmoid'))

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

cnn_history = cnn_model.fit(
    X_train_padded, y_train,
    epochs=5,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stopping]
)

loss, accuracy = cnn_model.evaluate(X_test_padded, y_test)
print("📊 CNN Model Performansı")
print("Accuracy:", accuracy)

c:\Users\sagla\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 289s 254ms/step - accuracy: 0.7587 - loss: 0.4671 - val_accuracy: 0.8913 - val_loss: 0.2551
Epoch 2/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 271s 241ms/step - accuracy: 0.9255 - loss: 0.2008 - val_accuracy: 0.9010 - val_loss: 0.2416
Epoch 3/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 270s 240ms/step - accuracy: 0.9649 - loss: 0.1104 - val_accuracy: 0.9032 - val_loss: 0.2534
Epoch 4/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 260s 231ms/step - accuracy: 0.9901 - loss: 0.0470 - val_accuracy: 0.9045 - val_loss: 0.2832
Epoch 5/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 263s 234ms/step - accuracy: 0.9981 - loss: 0.0166 - val_accuracy: 0.9005 - val_loss: 0.3475
313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 46ms/step - accuracy: 0.8967 - loss: 0.2490
📊 CNN Model Performansı
Accuracy: 0.8970000147819519


In [10]:
from sklearn.metrics import accuracy_score, f1_score

# Tahminler (0 veya 1)
y_pred = (cnn_model.predict(X_test_padded) > 0.5).astype("int32")

# Skorlar
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {acc:.4f}")
print(f"F1 Score: {f1:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 48ms/step
Accuracy: 0.8970
F1 Score: 0.8971


In [9]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping

# LSTM modelini oluşturma
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=max_sequence_length))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))

# Modeli derleme
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping tanımlama
early_stopping = EarlyStopping(
    monitor='val_loss',  # İzlenecek metrik
    patience=3,          # Doğrulama kaybı iyileşmediğinde kaç epoch bekleyeceği
    restore_best_weights=True  # En iyi ağırlıkları geri yükle
)

# Modeli eğitme
history = model.fit(
    X_train_padded, y_train,
    epochs=5,  # Eğitim süresini uzatabiliriz çünkü early stopping ile erken durdurma yapılacak
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stopping]  # Early stopping'i geri çağırmalara ekleyin
)

# Modeli değerlendirme
loss, accuracy = model.evaluate(X_test_padded, y_test)
print("LSTM Model Performansı")
print("Accuracy:", accuracy)

c:\Users\sagla\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 911s 807ms/step - accuracy: 0.7323 - loss: 0.5144 - val_accuracy: 0.7620 - val_loss: 0.5037
Epoch 2/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 892s 793ms/step - accuracy: 0.8731 - loss: 0.3135 - val_accuracy: 0.8687 - val_loss: 0.3392
Epoch 3/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 899s 799ms/step - accuracy: 0.9035 - loss: 0.2438 - val_accuracy: 0.8795 - val_loss: 0.2882
Epoch 4/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 941s 837ms/step - accuracy: 0.9189 - loss: 0.2070 - val_accuracy: 0.8775 - val_loss: 0.3028
Epoch 5/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 995s 885ms/step - accuracy: 0.9391 - loss: 0.1639 - val_accuracy: 0.8830 - val_loss: 0.3155
313/313 ━━━━━━━━━━━━━━━━━━━━ 63s 200ms/step - accuracy: 0.8915 - loss: 0.2741
LSTM Model Performansı
Accuracy: 0.8913999795913696


In [10]:
from sklearn.metrics import accuracy_score, f1_score

# Tahminler (0 veya 1)
y_pred = (model.predict(X_test_padded) > 0.5).astype("int32")

# Skorlar
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {acc:.4f}")
print(f"F1 Score: {f1:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 63s 200ms/step
Accuracy: 0.8914
F1 Score: 0.8951


In [ ]:
"""import tensorflow as tf
from sklearn.metrics import classification_report

def train_and_evaluate(model_type):
    print(f"\n📚 Training {model_type.upper()} model...")

    # Modeli oluştur
    model = create_model(model_type)

    # En iyi modeli kaydetmek için callback
    # 'val_accuracy' metrikte en iyi (maksimum) değeri gördüğünde kaydeder
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=f'best_{model_type}.keras',  # Dosya adı
        monitor='val_accuracy',           # izlenecek metrik
        mode='max',                           # en yüksek metrik değerini seç
        save_best_only=True,             # sadece en iyi modeli sakla
        verbose=1
    )

    # Eğitim
    history = model.fit(
        x_train_pad, y_train,
        epochs=10,
        batch_size=128,
        validation_split=0.2,
        verbose=1,
        callbacks=[checkpoint_callback]   # callback listesi
    )

    # Eğitim bittiğinde 'best_{model_type}.h5' dosyasında en iyi epoch modeli var
    # Şimdi o en iyi modeli yükleyelim
    best_model = tf.keras.models.load_model(f'best_{model_type}.keras')

    # Test verisinde tahmin
    y_pred = (best_model.predict(x_test_pad) > 0.5).astype("int32")

    # Rapor
    print(f"\n📊 {model_type.upper()} Classification Report (Best Epoch):")
    print(classification_report(y_test, y_pred))"""


In [ ]:
#for m in ['cnn', 'rnn', 'lstm','gru']:
#    train_and_evaluate(m)


📚 Training CNN model...


c:\Users\sagla\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
249/250 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.6300 - loss: 0.5958
Epoch 1: val_accuracy improved from -inf to 0.85025, saving model to best_cnn.keras
250/250 ━━━━━━━━━━━━━━━━━━━━ 15s 52ms/step - accuracy: 0.6310 - loss: 0.5949 - val_accuracy: 0.8503 - val_loss: 0.3467
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.8896 - loss: 0.2723
Epoch 2: val_accuracy improved from 0.85025 to 0.86225, saving model to best_cnn.keras
250/250 ━━━━━━━━━━━━━━━━━━━━ 13s 51ms/step - accuracy: 0.8896 - loss: 0.2723 - val_accuracy: 0.8622 - val_loss: 0.3324
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.9463 - loss: 0.1499
Epoch 3: val_accuracy did not improve from 0.86225
250/250 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - accuracy: 0.9463 - loss: 0.1499 - val_accuracy: 0.8583 - val_loss: 0.3721
Epoch 4/10
249/250 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.9777 - loss: 0.0740
Epoch 4: val_accuracy did not improve from 0.86225
250/250 ━━━━━━━━━━━━━━

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# Tahminler (0 veya 1)
y_pred = (model.predict(X_test_padded) > 0.5).astype("int32")

# Skorlar
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {acc:.4f}")
print(f"F1 Score: {f1:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
✅ Accuracy: 0.8609
🎯 F1-score: 0.8665
313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 47ms/step
✅ Accuracy: 0.8740
🎯 F1-score: 0.8755
313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 44ms/step
✅ Accuracy: 0.8781
🎯 F1-score: 0.8806


In [11]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense

gru_model = Sequential()
gru_model.add(Embedding(input_dim=5000, output_dim=128, input_length=max_sequence_length))
gru_model.add(GRU(64))
gru_model.add(Dense(1, activation='sigmoid'))

gru_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

gru_history = gru_model.fit(
    X_train_padded, y_train,
    epochs=5,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stopping]
)

loss, accuracy = gru_model.evaluate(X_test_padded, y_test)
print("📊 GRU Model Performansı")
print("Accuracy:", accuracy)

c:\Users\sagla\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 909s 802ms/step - accuracy: 0.7148 - loss: 0.5246 - val_accuracy: 0.8845 - val_loss: 0.2989
Epoch 2/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 783s 696ms/step - accuracy: 0.9014 - loss: 0.2529 - val_accuracy: 0.8992 - val_loss: 0.2506
Epoch 3/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 800s 711ms/step - accuracy: 0.9339 - loss: 0.1828 - val_accuracy: 0.8990 - val_loss: 0.2667
Epoch 4/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 813s 722ms/step - accuracy: 0.9525 - loss: 0.1369 - val_accuracy: 0.8915 - val_loss: 0.2918
Epoch 5/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 817s 727ms/step - accuracy: 0.9666 - loss: 0.0981 - val_accuracy: 0.8832 - val_loss: 0.3120
313/313 ━━━━━━━━━━━━━━━━━━━━ 59s 186ms/step - accuracy: 0.9068 - loss: 0.2439
📊 GRU Model Performansı
Accuracy: 0.90420001745224


In [12]:
from sklearn.metrics import accuracy_score, f1_score

# Tahminler (0 veya 1)
y_pred = (gru_model.predict(X_test_padded) > 0.5).astype("int32")

# Skorlar
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {acc:.4f}")
print(f"F1 Score: {f1:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 58s 185ms/step
Accuracy: 0.9042
F1 Score: 0.9057


In [ ]:
from transformers import BertTokenizer
# BERT tokenizer yükle
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

c:\Users\sagla\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset

# 1) Veri Hazırlığı
# -----------------
# Örnek: df içindeki 'cleaned_review' (metin) ve 'label' (0,1) sütunlarını kullanıyoruz
# Elimizde: df['cleaned_review'], df['label']

# Küçük bir örneklik veriyi train/test olarak bölelim:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# 2) Tokenizer
# ------------
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts.reset_index(drop=True)
        self.labels = labels.reset_index(drop=True)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = int(self.labels[idx])
        encoding = tokenizer(text, truncation=True, padding='max_length', max_length=128)
        # PyTorch tensörleri:
        item = {key: torch.tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.tensor(label)
        return item

# Dataset nesneleri oluştur
train_dataset = TextDataset(train_df['cleaned_review'], train_df['label'])
val_dataset = TextDataset(val_df['cleaned_review'], val_df['label'])

# 3) Modeli Yükle
# ----------------
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# 4) TrainingArguments
# --------------------
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,            # Hızlı olması için 1 epoch
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy='epoch',   # her epoch sonunda validasyon
    logging_dir='./logs',
    logging_steps=10,
    save_strategy='epoch'
)

# 5) Trainer ve metrik
# --------------------
from sklearn.metrics import accuracy_score, classification_report

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\sagla\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>={ACCELERATE_MIN_VERSION}'`

In [5]:
# Bütün verileri string'e dönüştür (örnek: NaN varsa da string olarak işlenir)
x_train = [str(x) for x in x_train]
x_test = [str(x) for x in x_test]

In [6]:
# Tokenize et
train_encodings = tokenizer(x_train, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(x_test, truncation=True, padding=True, max_length=512)

In [7]:
from torch.utils.data import Dataset
class IMDBDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.labels[idx])
        }

# Dataset nesnelerini oluştur
train_dataset = IMDBDataset(train_encodings, y_train)
val_dataset = IMDBDataset(val_encodings, y_test)

In [10]:

import pandas as pd
import numpy as np
import torch
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset

print("Kurulum tamam. Gerekli kütüphaneler yüklendi.")

Kurulum tamam. Gerekli kütüphaneler yüklendi.


In [11]:
# Örneğin:
print("x_train.shape:", x_train.shape)
print("y_train.shape:", y_train.shape)
print("x_test.shape:", x_test.shape)
print("y_test.shape:", y_test.shape)

# Örnek ilk veri
print("İlk eğitim örneği:", x_train[0])
print("İlk eğitim etiketi:", y_train[0])

x_train.shape: (40000,)
y_train.shape: (40000,)
x_test.shape: (10000,)
y_test.shape: (10000,)
İlk eğitim örneği: thats kept asking many fight screaming match swearing general mayhem permeate minute comparison also stand think onedimensional character little depth virtually impossible care happens badly written cypher director hang multicultural belief topic done much better drama tv cinemai must confess I'm really one spotting bad performance film must said nichola burley heroine slutty best friend wasim zakir nasty bullying brother absolutely terrible do not know acting school graduated id apply full refund post haste samina awan lead role manages impress cast socalled british talent well probably never hear least thats hope next time hire different scoutanother intriguing thought hideously fashionable soundtrack featuring like snow patrol ian brown keane I'm bit music fan I'm familiar artist output did not recognise track movie apart omnipresent run bsides anyone get many many musica

In [18]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = int(self.labels[idx])
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        # Dönüş: (input_ids, attention_mask, label)
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [21]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score
import numpy as np

In [22]:
# x_train, y_train, x_test, y_test zaten elinde
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_dataset = TextDataset(x_train, y_train, tokenizer, max_len=128)
test_dataset  = TextDataset(x_test,  y_test,  tokenizer, max_len=128)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=8, shuffle=False)


In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [25]:
num_epochs = 1  # Hızlı deneme

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            labels=labels
        )
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_loss:.4f}")

    # Doğrulama (validation/test)
    model.eval()
    preds_list, true_list = [], []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits  # boyut (batch_size, 2)
            preds = torch.argmax(logits, dim=-1)

            preds_list.extend(preds.cpu().numpy())
            true_list.extend(labels.cpu().numpy())

    acc = accuracy_score(true_list, preds_list)
    print(f"Validation Accuracy: {acc:.4f}")


KeyboardInterrupt: 